## Import libraries (make sure that they are installed on your environment)

In [2]:
import requests
# Python library for pulling data out of HTML and XML files.

from bs4 import BeautifulSoup
# selenium sends the standard Python commands to different browsers, despite variation in their browser's design.
# Python APIs empower us to connect with the browser through Selenium.
# module provides all the WebDriver implementations.

from selenium import webdriver
import pandas as pd

import json  # import if you are converting to json file as your output
import os

## Install packages using pip in terminal or !pip in jupyternotebook cell or install using following code

In [3]:
package = "selenium"
try:
    __import__package
    print("imported")
except:
    os.system("pip install " + package)
    print("Successfully installed")

## In browser, right click, inspect element, go to network and check for data, and check for elements to understand how the data table looks

In [4]:
# create a list of all required links (from where you want to scrape data)
loopy = [
"https://data.bls.gov/cew/apps/table_maker/v4/table_maker.htm#type=6&year=2015&qtr=A&own=5&area=39000&supp=0",
"https://data.bls.gov/cew/apps/table_maker/v4/table_maker.htm#type=6&year=2018&qtr=A&own=5&area=39000&supp=0",
"https://data.bls.gov/cew/apps/table_maker/v4/table_maker.htm#type=6&year=2018&qtr=A&own=5&area=39001&supp=0",
"https://data.bls.gov/cew/apps/table_maker/v4/table_maker.htm#type=6&year=2015&qtr=A&own=5&area=39001&supp=0",
"https://data.bls.gov/cew/apps/table_maker/v4/table_maker.htm#type=6&year=2018&qtr=A&own=5&area=US000&supp=0",
"https://data.bls.gov/cew/apps/table_maker/v4/table_maker.htm#type=6&year=2015&qtr=A&own=5&area=US000&supp=0"
]

# CREATE A LOOP TO INTERATE THROUGH EACH ELEMENT IN loopy

for stateYear in loopy:
    baseUrl = stateYear
    baseYear = baseUrl[73:77]  # extract year string
    # extract code string (US000 for US, 39000 for Ohio, 39001 for Adam's County)
    baseCode = baseUrl[95:100]

    # To link browser and trigger browser to load the data, it requires chromedriver or other browser driver
    # Download chromeDriver from https://chromedriver.storage.googleapis.com/index.html?path=81.0.4044.138/
    # browser = webdriver.Chrome(executable_path=r"./chromedriver.exe")  # for windows
    browser = webdriver.Chrome(
        executable_path=r"./chromedriver")  # for mac, locate file path

    try:
        browser.get(baseUrl)
        page = BeautifulSoup(browser.page_source, 'html.parser')
        tbody = page.find(id='cewTableBody')
        # create headings to avoid hassle of getting the headings from web
        headings = ['NAICS_Sector', 'Annual_Establishments', 'Annual_Average_Employment', 'Total_Annual_Wages',
        'Annual_Average_Weekly_Wage','Annual_Wages_per_Employee', 'Annual_Average_Employment_Location_Quotient',
        'Total_Annual_Wages_Location_Quotient']

        table_data = []
        for tr in tbody.find_all("tr"):  # find all table row from tbody
            t_row = {}
            for td, th in zip(tr.find_all("td"), headings):
                t_row[th] = td.text.replace('\n', '').replace('$', '').strip() # replace $ sign, and new line with no space
            table_data.append(t_row)
        # data is already stored in table_data


        # convert list to dataframe
        table_data_df = pd.DataFrame(table_data)
        # make new dataframe with NAICS_sector column split where there is space
        # we want the code and name to be separate
        new = table_data_df["NAICS_Sector"].str.split(" ", 2, expand=True)
        # making separate columns from dataframe separated by space and combining
        table_data_df["NAICS_Sector_Code"] = new[0]+" "+new[1]
        # making separate last industry column from new data frame
        table_data_df["NAICS_Sector_Industry"] = new[2]
        # rearrange columns and overwriting on same dataframe
        table_data_df = table_data_df[[
            'NAICS_Sector_Code', 'NAICS_Sector_Industry', 'NAICS_Sector', 'Annual_Establishments',
            'Annual_Average_Employment', 'Total_Annual_Wages', 'Annual_Average_Weekly_Wage',
            'Annual_Wages_per_Employee', 'Annual_Average_Employment_Location_Quotient',
            'Total_Annual_Wages_Location_Quotient']]
            
        # table_data_df.drop(columns=["NAICS_Sector"], inplace=True)         # to drop the old columns

        fileName = baseCode+"_"+baseYear+".csv"         # create a file name with area code and year in csv
        path=r'./Web_scraped_raw_data'                  # create a folder path to store csv files

        # table_data_df.to_csv(path,fileName,index=False)  # write as csv file
        table_data_df.to_csv(os.path.join(path,fileName),index=False) # write as csv file to a specific folder


        # fileName = baseCode+"_"+baseYear+".json"      # to output as json file
        # with open(fileName, 'w') as fp:
        #     json.dump(table_data_df, fp)

        pass
    except Exception as e:
        print(e)
    finally:
        browser.quit()

In [5]:
# to convert jupyter notebook into html, run this code on terminal

# pip install nbconvert
# jupyter nbconvert --to html webscraping.ipynb